In [1]:
import sys
sys.path.append('../..')

In [2]:
import os
import psutil

import random
import math
from functools import partial

import torch 
from torch import optim
from torch.optim import lr_scheduler
from torch import nn
from torch.nn import functional as F

import multiprocessing.dummy as mp

from pytorch_lightning import Trainer
from pytorch_lightning.core import LightningModule
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger


from lib.schedulers import DelayedScheduler
from lib.datasets import (max_lbl_nums, actual_lbl_nums, 
                          patches_rgb_mean_av1, patches_rgb_std_av1, 
                          get_train_test_img_ids_split)
from lib.dataloaders import PatchesDataset, WSIPatchesDatasetRaw, WSIPatchesDummyDataloader
from lib.augmentations import augment_v1_clr_only, augment_empty_clr_only
from lib.losses import SmoothLoss
from lib.trainers import WSIModuleV1

from lib.models.unetv1 import get_model
from lib.models.features_map import FeaturesMap

from sklearn.metrics import cohen_kappa_score

from tqdm.auto import tqdm

import matplotlib.pyplot as plt

In [3]:
# import cv2
import numpy as np
# import pandas as pd
# from lib.datasets import patches_csv_path, patches_path
from lib.datasets import (patches_clean90_csv_path as patches_csv_path, patches_path,
                          patches_clean90_pkl_path as patches_pkl_path)
# from lib.dataloaders import imread, get_g_score_num, get_provider_num

In [4]:
train_img_ids, test_img_ids = get_train_test_img_ids_split()

test_img_ids[:4]

['e8baa3bb9dcfb9cef5ca599d62bb8046',
 '9b2948ff81b64677a1a152a1532c1a50',
 '5b003d43ec0ce5979062442486f84cf7',
 '375b2c9501320b35ceb638a3274812aa']

In [5]:
from lib.dataloaders import WSIPatchesDataloader, WSIPatchesDatasetRaw
from lib.utils import get_pretrained_model, get_features

In [6]:
batch_size = 8

In [7]:
train_batch_path = '/mnt/SSDData/pdata/processed/pretrained_64x8x8/train/{}/'
test_batch_path = '/mnt/SSDData/pdata/processed/pretrained_64x8x8/val/'

train_loader = WSIPatchesDummyDataloader(train_batch_path, precalc_epochs=11, batch_size=batch_size, shuffle=True)
val_loader = WSIPatchesDummyDataloader(test_batch_path, precalc_epochs=11, batch_size=batch_size, shuffle=False)

In [8]:
steps_in_epoh = 1

epochs = 90

warmup_epochs = 0
warmup_steps = 0


hparams = {
    'module': {
        'name': 'lib.trainers.WSIModuleV1',
        'params': {
            'model': {
                'name': 'lib.models.wsi_resnets.Resnet_512x1x1',
                'params': {
                    'backbone': 'resnet18',
                    'backbone_features': 512,
                    'classes': max_lbl_nums,
                    'features_do': 0,
                }
            },
        },
    },
    'batch_size': batch_size,
    'learning_rate': 0.001 * 16 / 256,
    'dataset': {
        'dataloader': 'dummy',
        'rgb_mean': patches_rgb_mean_av1,
        'rgb_std': patches_rgb_std_av1,
        'classes': max_lbl_nums,
        'precalc_epochs': 11,
        'train_test_split': {},
    },
    'optimizer': {
        'name': 'torch.optim.Adam',
        'params': {
            'weight_decay': 1e-4
        }
    },
    'scheduler': {
        'name': 'torch.optim.lr_scheduler.ExponentialLR',
        'params': {
            'gamma': 0.96,
        },
        'interval': 'epoch'
    },
    'loss': {
        'weights': {
            'reg': 1 / 2,
            'class': 9 / 2
        },
        'label_smoothing': 0.1
    },
    'warmup_steps': warmup_steps,
    'steps_in_epoh': steps_in_epoh,
    'epochs': epochs
}

In [9]:
steps_in_epoh = len(train_loader)

In [10]:
steps_in_epoh

1053

In [11]:
hparams['steps_in_batch'] = steps_in_epoh
if 'T_max' in hparams['scheduler']['params']:
    hparams['scheduler']['params']['T_max'] = (epochs * steps_in_epoh -
                                               warmup_steps)

In [12]:
# tmp[0].shape
# torch.Size([64, 300, 64, 8, 8])

In [13]:
class FeaturesMap(nn.Module):
    def __init__(self, use_dummy_feature, f_channels=512, max_height=70,
                 max_width=40, f_size=1):
        super().__init__()
        
        self.f_size = f_size
        self.max_height = max_height * self.f_size
        self.max_width = max_width * self.f_size
        self.f_channels = f_channels
        self.use_dummy_feature = use_dummy_feature
        
        if self.use_dummy_feature:
            self.backend_feature =\
                nn.Parameter(torch.full((self.f_channels, 1, 1), 0))

    def forward(self, features, ys, xs, validation=None):
        if validation is None:
            validation = not self.training

        b_sz = features.shape[0]

        if self.use_dummy_feature:
            f_map = self.backend_feature.expand(self.f_channels,
                                                self.max_height,
                                                self.max_width)
        else:
            f_map = torch.full((self.f_channels, self.max_height,
                                self.max_width), 0,
                               dtype=features.dtype, device=features.device)

        res_f_maps = []
        for b in range(b_sz):
            real_mask = ys[b] > -1

            min_y, max_y = (ys[b, real_mask].min().item(),
                            ys[b, real_mask].max().item())
            min_x, max_x = (xs[b, real_mask].min().item(),
                            xs[b, real_mask].max().item())
            
            height = (max_y - min_y + 1) * self.f_size
            width = (max_x - min_x + 1) * self.f_size

            tmp_f_map = torch.full((self.f_channels, height, width), -1,
                                   dtype=features.dtype,
                                   device=features.device)

            if self.f_size == 1:
                tmp_f_map[:, ys[b, real_mask] - min_y, xs[b, real_mask] - min_x] =\
                    features[b, :, real_mask]
            else:
                for n in range(real_mask.sum().item()):
                    t = features[b, n]
                    _y = ys[b, n] - min_y
                    _x = xs[b, n] - min_x                    
                    tmp_f_map[:, _y*self.f_size:(_y+1)*self.f_size, 
                                 _x*self.f_size:(_x+1)*self.f_size] = t

            if width > height:
                tmp_f_map = tmp_f_map.transpose(-1, -2)

            _, height, width = tmp_f_map.shape
            # print(height, width)

            h_dif = height - self.max_height
            w_dif = width - self.max_width

            if h_dif > 0:
                cut_top = (math.ceil(h_dif / 2) if validation else
                           random.randint(0, h_dif))
                cut_bottom = -(h_dif - cut_top)
                if cut_bottom >= 0:
                    cut_bottom = None
                pad_top, pad_bottom = 0, 0
            else:
                pad_top = (math.ceil(-h_dif / 2) if validation else
                           random.randint(0, -h_dif))
                pad_bottom = -h_dif - pad_top
                cut_top, cut_bottom = 0, None

            if w_dif > 0:
                cut_left = (math.ceil(w_dif / 2) if validation else
                            random.randint(0, w_dif))
                cut_right = -(w_dif - cut_left)
                if cut_right >= 0:
                    cut_right = None
                pad_right, pad_left = 0, 0
            else:
                pad_right = (math.ceil(-w_dif / 2) if validation else
                             random.randint(0, -w_dif))
                pad_left = -w_dif - pad_right
                cut_left, cut_right = 0, None

            if not validation:
                if random.random() > 0.5:
                    tmp_f_map = torch.flip(tmp_f_map, [-1])

                if random.random() > 0.5:
                    tmp_f_map = torch.flip(tmp_f_map, [-2])

            tmp_f_map = F.pad(tmp_f_map[:, cut_top:cut_bottom,
                                        cut_left:cut_right],
                              (pad_right, pad_left,
                               pad_top, pad_bottom), value=-1)

            real_2d_mask = (tmp_f_map != -1).all(dim=0)

            # print(real_2d_mask.shape, f_map.shape, tmp_f_map.shape)

            res_f_map = (~real_2d_mask) * f_map + real_2d_mask * tmp_f_map
            res_f_maps.append(res_f_map)

        return torch.stack(res_f_maps)

In [14]:
#features, ys, xs, provider, isup_grade, gleason_score = next(iter(train_loader))

In [15]:
#f_map = FeaturesMap(False, 64, 70, 40, f_size=8)

In [16]:
#device = torch.device('cuda:0')

In [17]:
# features, ys, xs = features.to(device), ys.to(device), xs.to(device)

In [18]:
# tmp = f_map(features, ys, xs)

In [19]:
import torchvision.models as models

class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd
    def forward(self, x):
        return self.lambd(x)

f_d_rate = 0.0
d_rate = 0.0

max_height = 70
max_width = 40

class MainModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.f_map = FeaturesMap(True, 64, max_height, max_width, f_size=8)
        
        # self.backbone = models.resnext50_32x4d(pretrained=False)
        self.backbone = models.resnet18(pretrained=False)
        self.backbone.conv1 = nn.Sequential(
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 1),
        )
        #self.backbone.conv1 = nn.Identity()
        # self.backbone.fc = nn.Linear(2048, 512)
        self.backbone.fc = nn.Linear(512, 512)
        self.backbone.maxpool = nn.Identity()
        
        self.reg_linear = nn.Linear(512, 1)
        self.class_linear = nn.Linear(512, max_lbl_nums)

    def forward(self, features, ys, xs):
        f_map = self.f_map(features, ys, xs)
        x = self.backbone(f_map)
        return self.reg_linear(x), self.class_linear(x)

model = MainModel()

/opt/conda/conda-bld/pytorch_1587428266983/work/aten/src/ATen/native/TensorFactories.cpp:361: UserWarning: Deprecation warning: In a future PyTorch release torch.full will no longer return tensors of floating dtype by default. Instead, a bool fill_value will return a tensor of torch.bool dtype, and an integral fill_value will return a tensor of torch.long dtype. Set the optional `dtype` or `out` arguments to suppress this warning.


In [20]:
# model.backbone

In [21]:
#from torchsummary import summary

In [22]:
#summary(model.backbone, (64, 70*8, 40*8), device='cpu')

In [23]:
module = WSIModuleV1(model, hparams, log_train_every_batch=False)

In [24]:
trainer = Trainer(max_epochs=hparams['epochs'], gpus=[1,], fast_dev_run=False, num_sanity_val_steps=0)

INFO:lightning:GPU available: True, used: True
INFO:lightning:CUDA_VISIBLE_DEVICES: [1]


In [25]:
trainer.fit(module, train_loader, val_loader)
trainer.save_checkpoint(os.path.join(trainer.checkpoint_callback.dirpath,
                                     "last.ckpt"))

INFO:lightning:
   | Name                                 | Type              | Params
-----------------------------------------------------------------------
0  | model                                | MainModel         | 11 M  
1  | model.f_map                          | FeaturesMap       | 64    
2  | model.backbone                       | ResNet            | 11 M  
3  | model.backbone.conv1                 | Sequential        | 12 K  
4  | model.backbone.conv1.0               | BatchNorm2d       | 128   
5  | model.backbone.conv1.1               | Conv2d            | 4 K   
6  | model.backbone.conv1.2               | BatchNorm2d       | 128   
7  | model.backbone.conv1.3               | ReLU              | 0     
8  | model.backbone.conv1.4               | Conv2d            | 4 K   
9  | model.backbone.conv1.5               | BatchNorm2d       | 128   
10 | model.backbone.conv1.6               | ReLU              | 0     
11 | model.backbone.conv1.7               | Conv2d          

[Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    initial_lr: 6.25e-05
    lr: 6.25e-05
    weight_decay: 0.0001
)] [{'interval': 'epoch', 'scheduler': <lib.schedulers.DelayedScheduler object at 0x7fdcafb7fe10>}]


/home/ruslan/anaconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:23: RuntimeWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  warnings.warn(*args, **kwargs)


/home/ruslan/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:2247: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."
INFO:lightning:Detected KeyboardInterrupt, attempting graceful shutdown...
